In [1]:

import sys, os
cdir = %pwd
sys.path.append(os.path.join(cdir, '..', 'python'))


In [2]:
from hue_show import *

In [85]:
!curl https://www.meethue.com/api/nupnp

[{"id":"001788fffe273a83","internalipaddress":"10.0.1.18"},{"id":"001788fffe0aa167","internalipaddress":"10.0.1.17"}]

In [17]:
from time import sleep
from phue import Bridge
import random
from numpy import zeros, ones
import numpy as np
import matplotlib.pyplot as plt
from tqdm import *
import requests as req
import pandas as pd
from rgb_cie import Converter

In [18]:
bridges = req.get('https://www.meethue.com/api/nupnp').json()

In [69]:
b = Bridge('81.161.138.239:7745')

POST Request to 81.161.138.239:7745/api timed out.
Traceback (most recent call last):
  File "/home/pi/.local/lib/python3.4/site-packages/phue.py", line 551, in connect
    self.username = config[self.ip]['username']
KeyError: '81.161.138.239:7745'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/pi/.local/lib/python3.4/site-packages/phue.py", line 459, in request
    connection.request(mode, address, data)
  File "/usr/lib/python3.4/http/client.py", line 1090, in request
    self._send_request(method, url, body, headers)
  File "/usr/lib/python3.4/http/client.py", line 1128, in _send_request
    self.endheaders(body)
  File "/usr/lib/python3.4/http/client.py", line 1086, in endheaders
    self._send_output(message_body)
  File "/usr/lib/python3.4/http/client.py", line 924, in _send_output
    self.send(msg)
  File "/usr/lib/python3.4/http/client.py", line 859, in send
    self.connect()
  File "/usr/lib/python3.4/ht

PhueRequestTimeout: (None, 'POST Request to 81.161.138.239:7745/api timed out.')

In [86]:
bri = []
for br in bridges:
    b = Bridge(br['internalipaddress'])
    b.connect
    bri.append(b)

ConnectionRefusedError: [Errno 111] Connection refused

In [87]:
b = Bridge(bridges[0]['internalipaddress'])

ConnectionRefusedError: [Errno 111] Connection refused

In [88]:
b = Bridge(bridges[1]['internalipaddress'])

In [41]:
ln = 1
speed = 20 # m/s
distance = 7 #m between each lamp
time = 7 / (speed * 1E3 / (60*60))

def light(name, r_,g_,b_):
    l = b.lights_by_name[name]
    rgbl(l, r_,g_,b_, brightness=254)

normal(b)    
for n in range(9):
    try:
        name = "P%d"%((ln+n)%(19)+1)
        l = b.lights_by_name[name]
        rgbl(l, 1.0,0.0,0.0, brightness=254)
        sleep(time)
        rgbl(l, 1.,1.,1., brightness=254)
        sleep(0.1)
    except Exception as e:
        print(e)


KeyError: 'P1'

In [44]:
b.lights_by_name

{}

### Search for lights

In [45]:
import json
import time

In [46]:
b1 = b1[0]

IndexError: list index out of range

In [49]:
IP = b.ip

In [50]:
get = lambda i, str_:req.get('http://'+bri[i].ip+'/api/'+bri[i].username+'/'+str_).json()
post = lambda i, str_, data:req.post('http://'+bri[i].ip+'/api/'+bri[i].username+'/'+str_, json.dumps(data)).json()
put = lambda i, str_,  data:req.put('http://'+bri[i].ip+'/api/'+bri[i].username+'/'+str_, json.dumps(data)).json()

Search for new lights

In [52]:
post(0,'lights',{})
post(1,'lights',{})

IndexError: list index out of range

In [53]:
get(0,'lights/new')
get(1,'lights/new')

IndexError: list index out of range

In [59]:
print('now there are %d lights'%(len(get(0,'lights'))))

IndexError: list index out of range

In [58]:
lamps = {}
for i in range(len(bri)):
    print('Bridge-%d'%(i))
    for k,v in get(i,'lights').items():
        #if 'Hue color' in v['name']:
        print(v['name'], v['state']['reachable'], v['uniqueid'])
        idl = v['uniqueid']
        if idl in lamps:
            lamps[idl][i] = v
        else:
            lamps[idl] = {i:v}
    time.sleep(2)

In [57]:
for k,v in lamps.items():
    if len(v)>1:
        print('0:',   v[0]['state']['reachable'], v[0]['name'], 
              '\n1:', v[1]['state']['reachable'], v[1]['name'], 
              '\n')

In [73]:
lamps2 = {}
for k, v in lamps.items():
    if 0 in v:
        if 'P' in v[0]['name']:
                lamps2[v[0]['name']] = v
    if 1 in v:
        if 'P' in v[1]['name']:
            lamps2[v[1]['name']] = v
        

In [60]:
lamps2['P1']

NameError: name 'lamps2' is not defined

In [61]:
all_lights = ["DF1430", "7AC331", "B055E9", "1913F7", "2D2DA6", "0F9E4E", "DDE088", "209C5A", "E2B400", "6878AA", "34DE81", "62F1E6", "D9953F", "2A75A7", "5D3F46", "132A13", "81D5FE", "CF5C37"]

In [62]:
len(all_lights[0:10])

10

In [63]:
import json

In [64]:
import time

In [65]:
for i in range(len(bri)):
    post(i,'lights', {"deviceid":all_lights[0:10]})
    time.sleep(2)
    post(i,'lights', {"deviceid":all_lights[10:]})
    time.sleep(2)
    

In [66]:
bri

[]

### PARTY MODE

In [66]:
for i in range(len(bri)):
    put(i, 'groups/0/action', 
        {"on": True,
         "hue": 2000,
        })
    time.sleep(1)

In [ ]:
class Controler(object):
    def __init__(self):
        self.targets = {}
        

In [73]:
b = Bridge(bridges[1]['internalipaddress'])
b.username

'sVzdqoRQzArjcl9Uw1-IXEEQUVCBsnQeVy-d4J0o'

In [84]:
req.get('http://10.0.1.18/api/sVzdqoRQzAjclw1-IXEEQUVCBsnQeVy-d4J0o/lights/new').json()

[{'error': {'address': '/lights',
   'description': 'unauthorized user',
   'type': 1}}]

In [90]:
ffhue1='80185a5af6f2a719125d712fc76dd3'
ffhue2='192.168.0.199'

In [91]:
req.get('http://localhost:4566/api/%s/lights'%(ffhue2))

ConnectionError: ('Connection aborted.', ConnectionRefusedError(111, 'Connection refused'))

In [92]:
'http://localhost:4566/api/%s/lights'%(ffhue2)

'http://localhost:4566/api/EDTtC-PA4mzs7vVjrR0VWRaH3P4yYRqUIa2Ws6yP/lights'